## Task2

In [68]:
import numpy as np
import pandas as pd

import contractions
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

from sklearn.model_selection import train_test_split
import itertools, re
from collections import Counter 

from sklearn.naive_bayes import CategoricalNB, ComplementNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
# from sklearn.preprocessing import MultiLabelBinarizer
# mlb = MultiLabelBinarizer()


from sklearn.metrics import classification_report
# Scores
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import  recall_score


import warnings
warnings.filterwarnings("ignore")

In [2]:
def load_task2():
    temp = open('dontpatronizeme_categories.tsv')
    check = temp.read()
    tag2id = {'Unbalanced_power_relations':0,'Shallow_solution':1,'Presupposition':2,'Authority_voice':3,'Metaphors':4,
              'Compassion':5,'The_poorer_the_merrier':6}
    ls = check.split('\n')[4:]
    df = pd.DataFrame([line.split('\t')  for line in ls if len(line)>0])
    df.columns = ['par_id', 'art_id','text' ,'keyword', 'country','start','finish','text_span','label','num_annotators' ]
    df['labelid'] = df['label'].map(tag2id)
    return df

In [3]:
pcl_cat_df = load_task2()

In [152]:

# ## Not doing the mlb transformation. since I will use labelid column
# # pcl_cat_df.label = [list(i) for i in mlb.fit_transform([{i} for i in pcl_cat_df.label]) ]
# pcl_cat_df.labelid.value_counts()

In [63]:
pcl_cat_df

,par_id,art_id,text,keyword,country,start,finish,text_span,label,num_annotators,labelid,text_split,text_span_split,embeddings
0,4046,@@14767805,We also know that they can benefit by receivin...,hopeless,us,201,236,put their situation in perspective,Authority_voice,2,3,"[we, also, know, that, they, can, benefit, by,...","[put, their, situation, in, perspective]","[0.9346621, -5.8665924, -2.6035552, -3.6673388..."
1,4046,@@14767805,We also know that they can benefit by receivin...,hopeless,us,240,274,help them communicate with others,Unbalanced_power_relations,2,0,"[we, also, know, that, they, can, benefit, by,...","[help, them, communicate, with, others]","[1.0959765, -13.135075, -5.5036626, -7.898024,..."
2,4046,@@14767805,We also know that they can benefit by receivin...,hopeless,us,274,300,who could provide support,Unbalanced_power_relations,1,0,"[we, also, know, that, they, can, benefit, by,...","[who, could, provide, support]","[-3.9163299, -6.50406, -3.1795444, -10.035988,..."
3,4046,@@14767805,We also know that they can benefit by receivin...,hopeless,us,384,434,plan for their needs and the needs of their child,Authority_voice,2,3,"[we, also, know, that, they, can, benefit, by,...","[plan, for, their, needs, and, the, needs, of,...","[-0.5853478, -9.880554, 0.27753043, -9.771219,..."
4,4046,@@14767805,We also know that they can benefit by receivin...,hopeless,us,0,35,We also know that they can benefit,Unbalanced_power_relations,1,0,"[we, also, know, that, they, can, benefit, by,...","[we, also, know, that, they, can, benefit]","[5.664587, -4.916066, -14.801041, -7.9291306, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2755,10070,@@15573661,Fern ? ndez was a well-known philanthropist wh...,disabled,ng,0,210,Fern ? ndez was a well-known philanthropist wh...,Unbalanced_power_relations,1,0,"[fern, ?, ndez, was, a, well-known, philanthro...","[fern, ?, ndez, was, a, well-known, philanthro...","[14.108637, -35.828888, -15.737808, 3.5372179,..."
2756,6484,@@2559173,"Touched much by their plight , Commanding Offi...",homeless,lk,0,29,Touched much by their plight,Compassion,1,5,"[touched, much, by, their, plight, commanding,...","[touched, much, by, their, plight]","[2.3005762, -5.1412964, -4.6743145, -3.1482363..."
2757,6484,@@2559173,"Touched much by their plight , Commanding Offi...",homeless,lk,31,315,Commanding Officer and all ranks of the 7 Sri ...,Unbalanced_power_relations,1,0,"[touched, much, by, their, plight, commanding,...","[commanding, officer, and, all, ranks, of, the...","[-29.726833, 5.807111, 11.023658, -2.702696, 1..."
2758,6249,@@1947926,She reiterated her ministry 's commitment to p...,women,gh,153,205,gave a strong indication of hope for Ghanaian ...,Unbalanced_power_relations,1,0,"[she, reiterated, her, ministry, s, commitment...","[gave, a, strong, indication, of, hope, for, g...","[2.9761238, -5.6566, -8.207358, 4.8946347, 0.9..."


### PREPROCESSING

In [4]:
def contract_words(text):
        """
        Removes contractions from the text.
        """
        return contractions.fix(text)
def preprocess_text(text):
    """
    Should return a list of words
    """
    text = contract_words(text)
    text = text.lower()
#     text = text.replace('"', "").replace(",", "").replace("'", "")
    text = text.replace('"', "").replace(",", "").replace("'", "").replace("."," .") ## added by PAVAN
    
    ## To capture multiple # feature -- added by PAVAN
    if re.search(r'[a-z]+\#',text) :
        tmp_ls = text.split()
        text = ' '.join( [re.sub(pattern=r'\#',repl=' #',string=str(i)) if re.search(r'[a-z]+\#',str(i)) else i for i in tmp_ls])
        
    ## To capture # feature -- added by PAVAN
    if re.search(r'\#[a-z]+',text) :
        tmp_ls = text.split()
        text = ' '.join( [re.sub(pattern=r'\#',repl='hashtagfea ',string=str(i)) if re.search(r'\#[a-z]+',str(i)) else i for i in tmp_ls])
    return text.split()

In [5]:
preprocess_text('#thedignityproject')

['hashtagfea', 'thedignityproject']

In [6]:
preprocess_text('#ichh#thedignityproject')

['hashtagfea', 'ichh', 'hashtagfea', 'thedignityproject']

## FEATURE GENERATION - word2vec

In [8]:
class Word2VecModelTrainer:
    """
    Word2Vec Features. Basic Process: call train, call load.
    """

    def __init__(self, sentences=[], path="word2vec.wordvectors"):
        """
        params: 
        sentences: list of sentences (default: [])
        NOTE sentences should be like this 
        [["i", "am", "ready"],["some","other","sentence"]]
        """

        self.sentences = sentences
        self.path = path
    
    ## UPDATED version -- PAVAN
    def train(self, size=100, window=7):
        """
        Trains the model on sentences
        params: 
        size: size of the vector
        window: window size
        """
        
        model = Word2Vec(sentences=self.sentences, vector_size=size,
                         window=window, min_count=1, workers=4)
        model.save("word2vec_categories.model")
        
        model.build_vocab(corpus_iterable=self.sentences, update=True)
        
        model.train(corpus_iterable=self.sentences,
                        total_examples=model.corpus_count, epochs=30)

#         model.train(sentences=self.sentences,
#                     total_examples=model.corpus_count, epochs=30)
        word_vectors = model.wv
        word_vectors.save(self.path)
        return model, word_vectors

    def load_trained(self, path=""):
        """
        Loads the trained model
        """
        if(path != ""):
            load_path = path
        else:
            load_path = self.path
        wv = KeyedVectors.load(load_path, mmap='r')
        return wv

In [9]:
## Features
def get_bigrams(text_split):
    """
    :param text_split: split text (ref preprocessing).
    :return: list of bigrams
    """
    bigrams = [[text_split[i], text_split[i+1]]
               for i in range(len(text_split)-1)]

    return bigrams

def add_vectors(list_of_words, wv):
    """
    Returns sum of word vectors, use on text split (ref preprocessing)
    params:
    list_of_words: list of words
    wv: word2vec model
    """
    return sum([wv[word] for word in list_of_words])


In [10]:
## For creating word2vec model & embeddings -> I am using 'text' column. Since it has all the data
pcl_cat_df['text_split'] =pcl_cat_df['text'].apply(preprocess_text)

unique_sentences_ls =  pcl_cat_df['text_split'].to_list() 
# # Train WordVectors. Only run once.
Word2VecModelTrainer(sentences=unique_sentences_ls, path="word2vec_categories.wordvectors").train()


(<gensim.models.word2vec.Word2Vec at 0x1e1575656d0>,
 <gensim.models.keyedvectors.KeyedVectors at 0x1e157565af0>)

In [11]:
# Load the trained word vectors.
wv = Word2VecModelTrainer().load_trained("word2vec_categories.wordvectors")

pcl_cat_df['text_span_split'] =pcl_cat_df['text_span'].apply(preprocess_text)


## Get the treatment dictionary
def get_treatment_map(TEXT_SPAN_SPLIT_SERIES, TEXT_SPLIT_SERIES, EMBED):
    def custom_key(str):
        return -len(str), str.lower()
    text_span_token_ls = list(set( list( itertools.chain.from_iterable(TEXT_SPAN_SPLIT_SERIES) ))) #pcl_cat_df.text_span_split
    text_token_ls = list(set( list( itertools.chain.from_iterable( TEXT_SPLIT_SERIES ) ))) #pcl_cat_df.text_split
    TOBE_TREATED = [token for token in text_span_token_ls if token not in text_token_ls]

    sorted_token_ls = sorted(sorted(EMBED.key_to_index.keys() ), key=custom_key)
    EMBED_sorted = {i:EMBED[i] for i in sorted_token_ls}
    TREATING_TOKENS_MAP ={i:j for i in TOBE_TREATED for j in EMBED_sorted if re.match(pattern=i,string=j) }
    return TREATING_TOKENS_MAP

TREATING_TOKENS_MAP = get_treatment_map(pcl_cat_df.text_span_split,pcl_cat_df.text_split , wv)

## Applying the Treatment on "text_span_split" column
pcl_cat_df['text_span_split'] = pcl_cat_df['text_span_split'].apply(lambda x: [TREATING_TOKENS_MAP[i] if i in TREATING_TOKENS_MAP else i for i in x])

pcl_cat_df['embeddings'] = pcl_cat_df['text_span_split'].apply(add_vectors, wv=wv) #basic_features.


In [25]:
# define oversampling strategy
rus = RandomOverSampler(sampling_strategy='minority',random_state=42)
# fit and apply the transform
X_over, y_over = rus.fit_resample(X_train, y_train)
# summarize class distribution
print(Counter(y_over))

Counter({0: 774, 6: 774, 5: 546, 2: 237, 3: 228, 4: 200, 1: 181})


In [24]:
print(Counter(y_train))

Counter({0: 774, 5: 546, 2: 237, 3: 228, 4: 200, 1: 181, 6: 42})


## MODEL VERSION 1 - word2vec- NaiveBayes

In [51]:
## Use 'labelid' column for using CategoricalNB


X_train, X_test, y_train, y_test = train_test_split(
    pcl_cat_df['embeddings'], pcl_cat_df['labelid'], stratify=pcl_cat_df['labelid'], test_size=0.2, random_state=1)


# # define oversampling strategy
# rus = RandomUnderSampler(sampling_strategy= 'majority',random_state=42) #
# # fit and apply the transform
# X_over, y_over = rus.fit_resample(np.array(list(X_train)),np.array(list(y_train)) )

# X_train = X_over.copy()
# y_train = y_over.copy()

# y_train = y_train.sort_values()
# SORTED_INDICES= list(y_train.sort_values().index)
# X_train = X_train.loc[SORTED_INDICES]


scaler = MinMaxScaler()
X_train = scaler.fit_transform( np.array(list(X_train)) )
X_test = scaler.fit_transform( np.array(list(X_test)) )

model_NB = ComplementNB()
model_NB.fit(X_train,y_train)
pred_NB = model_NB.predict(X_test)

In [52]:
print(classification_report(y_true= y_test, y_pred= pred_NB))

              precision    recall  f1-score   support

           0       0.62      0.65      0.64       194
           1       0.00      0.00      0.00        46
           2       0.14      0.08      0.10        59
           3       0.16      0.40      0.22        57
           4       0.20      0.04      0.07        50
           5       0.48      0.53      0.50       136
           6       0.00      0.00      0.00        10

    accuracy                           0.41       552
   macro avg       0.23      0.24      0.22       552
weighted avg       0.38      0.41      0.39       552



### Feature Generation: TF-IDF

In [57]:
corpus = pcl_cat_df.text_split.apply(lambda x: ' '.join(x) ).drop_duplicates().to_list()

vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
X = vectorizer.transform(pcl_cat_df.text_span)

In [83]:
pcl_textspan_tfidf = pd.DataFrame(X.toarray(),columns= vectorizer.get_feature_names())
pca = PCA(n_components=100,random_state=123)

pcl_cat_df['tfidf_embeddings'] = list(  list( pca.fit_transform(pcl_textspan_tfidf) )) 


## MODEL VERSION 2 - tfidf- NaiveBayes

In [90]:
X_train, X_test, y_train, y_test = train_test_split(
    pcl_cat_df['tfidf_embeddings'], pcl_cat_df['labelid'], stratify=pcl_cat_df['labelid'], test_size=0.2, random_state=1)


# define oversampling strategy
rus = RandomOverSampler(sampling_strategy= 'minority',random_state=42) #
# fit and apply the transform
X_over, y_over = rus.fit_resample(np.array(list(X_train)),np.array(list(y_train)) )

X_train = X_over.copy()
y_train = y_over.copy()

# y_train = y_train.sort_values()
# SORTED_INDICES= list(y_train.sort_values().index)
# X_train = X_train.loc[SORTED_INDICES]

scaler = MinMaxScaler()
X_train = scaler.fit_transform( np.array(list(X_train)) )
X_test = scaler.fit_transform( np.array(list(X_test)) )

model_NB = ComplementNB()
model_NB.fit(X_train,y_train)
pred_NB = model_NB.predict(X_test)

print(classification_report(y_true= y_test, y_pred= pred_NB))

              precision    recall  f1-score   support

           0       0.54      0.58      0.56       194
           1       0.26      0.11      0.15        46
           2       0.26      0.10      0.15        59
           3       0.19      0.07      0.10        57
           4       0.26      0.36      0.30        50
           5       0.52      0.38      0.43       136
           6       0.02      0.20      0.03        10

    accuracy                           0.36       552
   macro avg       0.29      0.26      0.25       552
weighted avg       0.41      0.36      0.37       552



In [128]:
## Preprocessing examples

## Points to be noted:
## 892 index: Text : #thedignityproject text_span = thedignityproject 
## -> To capture the essence of hashtag. I will replace #abc -> hashtagfea abc & get the hashtagfea into textspan so that the embedding of hashtag is present 

## Text : selected. text_span = selected
## -> To tackle this replaced '.' with ' . '

## hashtag example
pcl_cat_df.loc[ (pcl_cat_df['text_span_split'].apply(lambda x: ' '.join(x)).str.find('thedignityproject') != -1),'text' ].to_list()
pcl_cat_df.loc[ (pcl_cat_df['text_span_split'].apply(lambda x: ' '.join(x)).str.find('thedignityproject') != -1),'text_split' ].to_list()

## merged hashtag example
pcl_cat_df.loc[ pcl_cat_df['text_split'].apply(lambda x: 'hashtagfea' in x),'text' ].to_list()

## irregular clip - futur example
pcl_cat_df.loc[ (pcl_cat_df['text_span_split'].apply(lambda x: 'futur' in x)),'text_span' ]

Series([], Name: text_span, dtype: object)